In [1]:
import json

In [9]:
#Not San Andres y Providencia
data = json.load(open('./geojsons/colombia/colombia.geojson'))
new_features = []
for f in data['features']:
    if f['properties']['DPTO'] != '88':
        new_features.append(f)
data['features'] = new_features
with open('./geojsons/colombia/colombia_not_s_y_p.geojson', 'w') as fp:
    json.dump(data, fp)

In [6]:
standard_dict = {    
    "ANTIOQUIA":"ANTIOQUIA",
    "ATLANTICO":"ATLANTICO",
    "SANTAFE DE BOGOTA D.C":"BOGOTA",
    "BOLIVAR":"BOLIVAR",
    "BOYACA":"BOYACA",
    "CALDAS":"CALDAS",
    "CAQUETA":"CAQUETA",
    "CAUCA":"CAUCA",
    "CESAR":"CESAR",
    "CORDOBA":"CORDOBA",
    "CUNDINAMARCA":"CUNDINAMARCA",
    "CHOCO":"CHOCO",
    "HUILA":"HUILA",
    "LA GUAJIRA":"GUAJIRA",
    "MAGDALENA":"MAGDALENA",
    "META":"META",
    "NARIÑO":"NARINO",
    "NORTE DE SANTANDER":"NORTEDESANTANDER",
    "QUINDIO":"QUINDIO",
    "RISARALDA":"RISARALDA",
    "SANTANDER":"SANTANDER",
    "SUCRE":"SUCRE",
    "TOLIMA":"TOLIMA",
    "VALLE DEL CAUCA":"VALLEDELCAUCA",
    "ARAUCA":"ARAUCA",
    "CASANARE":"CASANARE",
    "PUTUMAYO":"PUTUMAYO",
    "AMAZONAS":"AMAZONAS",
    "GUAINIA":"GUAINIA",
    "GUAVIARE":"GUAVIARE",
    "VAUPES":"VAUPES",
    "VICHADA":"VICHADA",
    "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA":"SANANDRES"
}

In [9]:
#Standard names
data = json.load(open('./geojsons/colombia/colombia.geojson'))
new_features = []
for f in data['features']:
    f['properties']['STD_ID'] = standard_dict[f['properties']['NOMBRE_DPT']]
    new_features.append(f)
data['features'] = new_features
with open('./geojsons/colombia/colombia_std_names.geojson', 'w') as fp:
    json.dump(data, fp)